# Visualization Renderer

The Draco specification format was designed with extensibility and renderer-agnosticity in mind. Nevertheless, we ship the package with a default renderer implementation, based on [Vega-Altair](https://altair-viz.github.io/), taking advantage of [Vega](https://vega.github.io/vega) and [Vega-Lite](https://vega.github.io/vega-lite). The renderer shall facilitate debugging as well as it shall prove to be useful when integrating Draco into applications with a frontend.

```{note}
This chapter serves to provide a brief overview about the **usage** of the renderer. If you are curious about the implementation details, you can look into the [renderer API docs](../api/renderer).
```

## Usage

In order to render visualizations with Draco, you need two things:

- A specification describing a visualization either in a dictionary-based or an ASP-based format
- A Pandas `DataFrame` of data to be visualized

In what follows, we provide examples for both specification formats using a very simple dataset.

In [21]:
# Suppressing warnings raised by altair in the background
# (iteration-related deprecation warnings)
import warnings

warnings.filterwarnings("ignore")

In [22]:
import pandas as pd

values = [
    {"a": "A", "b": 28}, {"a": "B", "b": 55}, {"a": "C", "b": 43},
    {"a": "D", "b": 91}, {"a": "E", "b": 81}, {"a": "F", "b": 53},
    {"a": "G", "b": 19}, {"a": "H", "b": 87}, {"a": "I", "b": 52}
]
df = pd.DataFrame(values)
df.head()

,a,b
0,A,28
1,B,55
2,C,43
3,D,91
4,E,81


### Rendering a dictionary-based specification

In [23]:
from draco.renderer import AltairRenderer
from draco.schema import schema_from_dataframe

# Generating the data schema to extract the field types automatically
schema: dict = schema_from_dataframe(df)

# Defining a bar chart
bar_chart = {
    "view": [
        {
            "coordinates": "cartesian",
            "mark": [
                {
                    "type": "bar",
                    "encoding": [
                        {
                            "channel": "x",
                            "field": "a",
                        },
                        {
                            "channel": "y",
                            "field": "b",
                        }
                    ]
                }
            ],
            "scale": [
                {"channel": "x", "type": "ordinal"},
                {"channel": "y", "type": "linear", "zero": "true"},
            ]
        }
    ]
}
# Joining the data `schema` dict with the view specification dict
spec = schema | bar_chart

renderer = AltairRenderer()
renderer.render(spec, df)

alt.Chart(...)

### Rendering an ASP-based specification

In [24]:
from draco.fact_utils import dict_to_facts, answer_set_to_dict
from draco.run import run_clingo

# Re-using the spec from the previous example, transforming it into an ASP program
program = dict_to_facts(spec)

# Solving the program and transforming the answer set to a dict,
# so that it can be plugged into the renderer
answer_set = next(run_clingo(program)).answer_set
spec = answer_set_to_dict(answer_set)

renderer = AltairRenderer()
renderer.render(spec, df)

alt.Chart(...)

```{caution}
The default renderer only handles the specification format as described in the [Facts](../facts/intro) chapters. If you modify or extend this format you will need to implement your own renderer accordingly. You can read more about the technical details of the renderer in the [renderer API docs](../api/renderer).
```

## More Examples

For more complex examples, you can refer to the [Example Charts](../facts/examples.ipynb) chapter.